In [1]:
data = r"C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\IR\UU + Perpu.csv"
stopword_id = r'C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\IR\stopwords-id.txt'

In [2]:
import pandas as pd
import numpy as np
import string
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from joblib import Parallel, delayed
from joblib import Memory
from sklearn.decomposition import TruncatedSVD

# Caching setup
location = './cacheall'
memory = Memory(location, verbose=0)

# Inisialisasi stemmer bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Membaca daftar stop words bahasa Indonesia
# stopword_id = 'stopwords.txt'  # Specify the path to your stopwords file
with open(stopword_id, 'r') as f:
    stop_words_id = f.read().splitlines()

def preprocess_text(text):
    # Pastikan text adalah string
    if not isinstance(text, str):
        text = str(text)
    
    # Menghilangkan karakter berulang
    text = re.sub(r'(.)\1+', r'\1', text)
    
    # Menghilangkan angka
    text = ''.join([i for i in text if not i.isdigit()])
    
    # Menghilangkan tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Mengubah teks menjadi huruf kecil
    text = text.lower()
    
    # Melakukan stemming pada teks
    text = stemmer.stem(text)
    
    return text

@memory.cache
def preprocess_parallel(text_series):
    return Parallel(n_jobs=-1)(delayed(stemmer.stem)(text) for text in text_series)

# Membaca file CSV
# data = 'data.csv'  # Specify the path to your CSV file
df = pd.read_csv(data)

# Pastikan tidak ada nilai NaN dan semua nilai adalah string
df['Teks'] = preprocess_parallel(df['Teks'].fillna(''))

# Inisialisasi TfidfVectorizer dengan stop words bahasa Indonesia
tfidf = TfidfVectorizer(stop_words=stop_words_id, max_df=0.85, min_df=2, ngram_range=(1, 2))

# Melakukan fit dan transformasi pada kolom Teks
tfidf_matrix = tfidf.fit_transform(df['Teks'])

# Menggunakan TruncatedSVD untuk pengurangan dimensi
svd = TruncatedSVD(n_components=100)
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)

def search_documents(query, top_n=10):
    # Preprocessing query
    query = preprocess_text(query)
    
    # Transformasi query menjadi vektor tf-idf
    query_vec = tfidf.transform([query])
    query_vec_reduced = svd.transform(query_vec)
    
    # Menghitung cosine similarity antara query dan semua dokumen
    cosine_similarities = linear_kernel(query_vec_reduced, tfidf_matrix_reduced).flatten()
    
    # Mendapatkan indeks dokumen dengan similarity tertinggi
    related_docs_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Mendapatkan judul, teks, dan nilai similarity dari dokumen yang relevan
    results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i], i) for i in related_docs_indices]
    
    return results

def get_feedback(results):
    feedback = []
    for idx, (title, text, similarity, doc_index) in enumerate(results):
        print(f"Dokumen {idx + 1}:")
        print(f"Judul: {title}")
        print(f"Teks: {text[:200]}...")  # Display only the first 200 characters
        print(f"Similarity: {similarity:.4f}")
        relevansi = int(input("Masukkan nilai relevansi (1-5): "))
        feedback.append((doc_index, relevansi))
    return feedback

def optimize_with_feedback(feedback, tfidf_matrix_reduced):
    relevant_docs = [idx for idx, relevansi in feedback if relevansi >= 3]
    non_relevant_docs = [idx for idx, relevansi in feedback if relevansi < 3]
    
    if not relevant_docs:
        print("Tidak ada dokumen yang dianggap relevan. Pencarian ulang tidak dapat dilakukan.")
        return None
    
    relevant_matrix = tfidf_matrix_reduced[relevant_docs]
    non_relevant_matrix = tfidf_matrix_reduced[non_relevant_docs] if non_relevant_docs else np.zeros(relevant_matrix.shape)
    
    # Compute the centroid of relevant and non-relevant documents
    relevant_centroid = np.asarray(relevant_matrix.mean(axis=0)).flatten()
    non_relevant_centroid = np.asarray(non_relevant_matrix.mean(axis=0)).flatten() if non_relevant_docs else np.zeros(relevant_centroid.shape)
    
    # Update query vector by moving it towards the relevant centroid and away from the non-relevant centroid
    def adjust_query_vec(query_vec, relevant_centroid, non_relevant_centroid, alpha=1, beta=0.75, gamma=0.15):
        return alpha * query_vec + beta * relevant_centroid - gamma * non_relevant_centroid
    
    return adjust_query_vec

# Example usage
if __name__ == "__main__":
    query = input("Masukkan Query: ")
    print(f"Query: {query}\n")
    print()
    initial_results = search_documents(query)
    
    feedback = get_feedback(initial_results)
    adjust_query_vec = optimize_with_feedback(feedback, tfidf_matrix_reduced)
    
    if adjust_query_vec:
        relevant_docs = [idx for idx, relevansi in feedback if relevansi >= 3]
        non_relevant_docs = [idx for idx, relevansi in feedback if relevansi < 3]
        
        # Reprocess the query with the adjusted query vector
        query_vec = tfidf.transform([preprocess_text(query)])
        query_vec_reduced = svd.transform(query_vec)
        adjusted_query_vec = adjust_query_vec(query_vec_reduced, np.asarray(tfidf_matrix_reduced[relevant_docs].mean(axis=0)).flatten(), 
                                              np.asarray(tfidf_matrix_reduced[non_relevant_docs].mean(axis=0)).flatten() if non_relevant_docs else np.zeros(query_vec_reduced.shape))
        
        # Compute cosine similarity with the adjusted query vector
        cosine_similarities = linear_kernel(adjusted_query_vec, tfidf_matrix_reduced).flatten()
        related_docs_indices = cosine_similarities.argsort()[-10:][::-1]
        
        # Display optimized results
        optimized_results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i]) for i in related_docs_indices]
        print("\n\n -- HASIL PENELUSURAN ULANG -- \n\n")
        for idx, (title, text, similarity) in enumerate(optimized_results):
            print(f"Dokumen {idx + 1}:")
            print(f"Judul: {title}")
            print(f"Similarity: {similarity:.4f}")
            print()


Query: pendidikan


Dokumen 1:
Judul: Undang-Undang Nomor 2 Tahun 1989
Teks: undang republik indonesia nomor 2 tahun 1989 tentang sistem didik nasional dengan rahmat tuhan yang maha esa presiden republik indonesia timbang a bahwa undang dasar 1945 amanat upaya untuk cerdas hid...
Similarity: 0.4045
Dokumen 2:
Judul: Undang-Undang Nomor 20 Tahun 2003
Teks: teks tidak dalam format asli lembar negara republik indonesia undang republik indonesia nomor 20 tahun 2003 tentang sistem didik nasional dengan rahmat tuhan yang maha esa presiden republik indonesia ...
Similarity: 0.4028
Dokumen 3:
Judul: Undang-Undang Nomor 12 Tahun 1954
Teks: undang republik indonesia nomor 12 tahun 1954 tentang nyata laku undang nr 4 tahun 1950 dari republik indonesia dahulu tentang dasar didik dan ajar di sekolah untuk seluruh indonesia presiden republik...
Similarity: 0.3169
Dokumen 4:
Judul: Undang-Undang Nomor 12 Tahun 2012
Teks: lembar negara republik indonesia no 158 2012 didik didik tinggi perintah perinta

In [2]:
import pandas as pd
import numpy as np
import string
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from joblib import Parallel, delayed
from joblib import Memory

# Caching setup
location = './cacheall'
memory = Memory(location, verbose=0)

# Inisialisasi stemmer bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Membaca daftar stop words bahasa Indonesia
with open(stopword_id, 'r') as f:
    stop_words_id = f.read().splitlines()

def preprocess_text(text):
    # Pastikan text adalah string
    if not isinstance(text, str):
        text = str(text)
    
    # Menghilangkan karakter berulang
    text = re.sub(r'(.)\1+', r'\1', text)
    
    # Menghilangkan angka
    text = ''.join([i for i in text if not i.isdigit()])
    
    # Menghilangkan tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Mengubah teks menjadi huruf kecil
    text = text.lower()
    
    # Melakukan stemming pada teks
    text = stemmer.stem(text)
    
    return text

@memory.cache
def preprocess_parallel(text_series):
    return Parallel(n_jobs=-1)(delayed(stemmer.stem)(text) for text in text_series)


# Membaca file CSV
df = pd.read_csv(data)

# # Pastikan tidak ada nilai NaN dan semua nilai adalah string
# df['Teks'] = preprocess_parallel(df['Teks'].fillna(''))

# Inisialisasi TfidfVectorizer dengan stop words bahasa Indonesia
tfidf = TfidfVectorizer(stop_words=stop_words_id, max_df=0.85, min_df=2, ngram_range=(1, 2))

# Melakukan fit dan transformasi pada kolom Teks
tfidf_matrix = tfidf.fit_transform(df['Teks'])

def search_documents(query, top_n=10):
    # Preprocessing query
    query = preprocess_text(query)
    
    # Transformasi query menjadi vektor tf-idf
    query_vec = tfidf.transform([query])
    
    # Menghitung cosine similarity antara query dan semua dokumen
    cosine_similarities = linear_kernel(query_vec, tfidf_matrix).flatten()
    
    # Mendapatkan indeks dokumen dengan similarity tertinggi
    related_docs_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Mendapatkan judul, teks, dan nilai similarity dari dokumen yang relevan
    results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i], i) for i in related_docs_indices]
    
    return results

def get_feedback(results):
    feedback = []
    for idx, (title, text, similarity, doc_index) in enumerate(results):
        print(f"Dokumen {idx + 1}:")
        print(f"Judul: {title}")
        print(f"Teks: {text[:200]}...")  # Display only the first 200 characters
        print(f"Similarity: {similarity:.4f}")
        relevansi = int(input("Masukkan nilai relevansi (1-5): "))
        feedback.append((doc_index, relevansi))
    return feedback

def optimize_with_feedback(feedback, tfidf_matrix):
    relevant_docs = [idx for idx, relevansi in feedback if relevansi >= 3]
    non_relevant_docs = [idx for idx, relevansi in feedback if relevansi < 3]
    
    if not relevant_docs:
        print("Tidak ada dokumen yang dianggap relevan. Pencarian ulang tidak dapat dilakukan.")
        return None
    
    relevant_matrix = tfidf_matrix[relevant_docs]
    non_relevant_matrix = tfidf_matrix[non_relevant_docs] if non_relevant_docs else np.zeros(relevant_matrix.shape)
    
    # Compute the centroid of relevant and non-relevant documents
    relevant_centroid = np.asarray(relevant_matrix.mean(axis=0)).flatten()
    non_relevant_centroid = np.asarray(non_relevant_matrix.mean(axis=0)).flatten() if non_relevant_docs else np.zeros(relevant_centroid.shape)
    
    # Update query vector by moving it towards the relevant centroid and away from the non-relevant centroid
    def adjust_query_vec(query_vec, relevant_centroid, non_relevant_centroid, alpha=1, beta=0.75, gamma=0.15):
        return alpha * query_vec + beta * relevant_centroid - gamma * non_relevant_centroid
    
    return adjust_query_vec

# Example usage
if __name__ == "__main__":
    query = input("Masukkan Query: ")
    print(f"Query: {query}\n")
    print()
    initial_results = search_documents(query)
    
    feedback = get_feedback(initial_results)
    adjust_query_vec = optimize_with_feedback(feedback, tfidf_matrix)
    
    if adjust_query_vec:
        relevant_docs = [idx for idx, relevansi in feedback if relevansi >= 3]
        non_relevant_docs = [idx for idx, relevansi in feedback if relevansi < 3]
        
        # Reprocess the query with the adjusted query vector
        query_vec = tfidf.transform([preprocess_text(query)])
        adjusted_query_vec = adjust_query_vec(query_vec.toarray(), np.asarray(tfidf_matrix[relevant_docs].mean(axis=0)).flatten(), 
                                              np.asarray(tfidf_matrix[non_relevant_docs].mean(axis=0)).flatten() if non_relevant_docs else np.zeros(query_vec.shape))
        
        # Compute cosine similarity with the adjusted query vector
        cosine_similarities = linear_kernel(adjusted_query_vec, tfidf_matrix).flatten()
        related_docs_indices = cosine_similarities.argsort()[-10:][::-1]
        
        # Display optimized results
        optimized_results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i]) for i in related_docs_indices]
        print("\n\n -- HASIL PENELUSURAN ULANG -- \n\n")
        for title, text, similarity in optimized_results:
            print(f"Judul: {title}")
            print(f"Similarity: {similarity:.4f}")
            print()

Query: pendidikan


Dokumen 1:
Judul: Undang-Undang Nomor 2 Tahun 1989
Teks:     UNDANG-UNDANG REPUBLIK INDONESIA NOMOR 2 TAHUN 1989 TENTANG  SISTEM PENDIDIKAN NASIONAL   DENGAN RAHMAT TUHAN YANG MAHA ESA  PRESIDEN REPUBLIK INDONESIA,    Menimbang  : a. bahwa Undang-Undang Das...
Similarity: 0.1878
Dokumen 2:
Judul: Undang-Undang Nomor 20 Tahun 2003
Teks: Teks tidak dalam format asli.   LEMBARAN NEGARA REPUBLIK INDONESIA    UNDANG-UNDANG REPUBLIK INDONESIA NOMOR 20 TAHUN 2003 TENTANG SISTEM PENDIDIKAN NASIONAL  DENGAN RAHMAT TUHAN YANG MAHA ESA  PRESID...
Similarity: 0.1725
Dokumen 3:
Judul: Undang-Undang Nomor 9 Tahun 2009
Teks: UNDANG-UNDANG REPUBLIK INDONESIA  NOMOR  9  TAHUN 2009 TENTANG  BADAN HUKUM PENDIDIKAN DENGAN RAHMAT TUHAN YANG MAHA ESA PRESIDEN REPUBLIK INDONESIA, Menimbang   :   a. bahwa untuk mewujudkan fungsi d...
Similarity: 0.0698
Dokumen 4:
Judul: Undang-Undang Nomor 12 Tahun 2010
Teks: UNDANG-UNDANG REPUBLIK INDONESIA NOMOR   12   TAHUN 200 TENTANG  GERAKAN PRAMUKA 